In [28]:
from Crypto.Util import number


RSA is based on the fact that multiplying p by q is easy, but factoring n 
is hard. The relation between the public (e) and the private (d) exponents is 
given by $\phi(n)$ that can only be calculated if you know p and q.
The correctness of the algorithm can be proven by Fermat's little theorem.

In RSA, we select two prime numbers of equal length (p and q), and then multiply these to give a modulus:

$ N=p*q $

with

$q<p<2q$

We then compute the cipher, encrypt a message $(M)$, as:

$ C=M^e\pmod n $

and where we decrypt with:

$ M=C^d\pmod n $

We then pick e so that it does not share a factor with  $\phi$:

$ GCD(e,\phi)=1 $

and where GCD is the greatest common denominator. In practice e typically has a value of 65537 (and which is a prime number), we could select a range of values of e. If we select a fairly large value, then the value of d could be discovered.

With this we have a public exponent of e, and a private exponent of d. The value of d is computed from:

$d=e^{−1}\pmod \phi$

and where

$ \phi(n)=(p−1)(q−1) $

it works when

$ M = C^d\pmod n = C^{-e}\pmod n$

thats gives

$de = 1 \pmod \phi$

that proofs decryption work

$C^d\pmod n = (M^e)^d \pmod n = M \pmod n$

In [29]:
# My secret message
msg = 'this is my secret'
# msg = 'Hello, 12345 is my secret number sequence'

data = msg.encode('utf-8')
M = number.bytes_to_long(data)

print("m:", msg)

m: this is my secret


## This is how the keys is generated

In [30]:
# Choose 2 large prime numbers p and q.
# For demo purposes to make print more readable
bitsize=256
p = number.getPrime(bitsize)
q = number.getPrime(bitsize)
# p = number.getStrongPrime(512)
# q = number.getStrongPrime(512)

# check they are not equal
assert(p != q)

# make sure that p is smaller than q
if p < q:
    (p, q) = (q, p)

# simple checks
assert(number.isPrime(p))
assert(number.isPrime(q))
assert(q < p)
assert(p < 2*q)

print("p:", p)
print("q:", q)
print("Prime size: %d bits" % (bitsize))


p: 79646538169893659311023099682874681527499439924354993686887701801717656561593
q: 62041720315356568690883921912206875856136496596522762564684551347896318131057
Prime size: 256 bits


In [31]:
# Setting up the encryption
# Find first part of the public key, n = p∗q.
n = p*q
phi = (p-1)*(q-1)


print("n:", n)
print("phi:", phi)
print("Encryption max size: %d bits" % (bitsize*2))
print("Encryption size: %d bits" % (n.bit_length()))


# simple check
assert(p*q == n)


n: 4941408245222913826115066046888904325684560320264209188239762831380104326702713764726239770396792159551303492824443440489374148043327378137455830566693801
phi: 4941408245222913826115066046888904325684560320264209188239762831380104326702572076467754520168790252529708411267059804552853270287075805884306216592001152
Encryption max size: 512 bits
Encryption size: 511 bits


### Key Generation

In [32]:
# generate public key 1<e<phi(n)
# The usual choice for e is F4=65537 = 0x10001. Also, having chosen e, it is simpler to test whether gcd(e,p-1)=1 and gcd(e,q-1)=1 while generating and testing the primes in step 1. Values of p or q that fail this test can be rejected there and then.
# https://www.di-mgt.com.au/rsa_alg.html#note2


e = 65537
# e = 0x10001
# e = number.getPrime(bitsize)
print("e:", e)

# "e" must be coprime with phi(n).
# You can check if e and phi(n) are coprimes using GCD(e, phi(n)) == 1.


assert(e > 1)
assert(number.GCD(e, n) == 1)
assert(number.GCD(e, p - 1) == 1)
assert(number.GCD(e, q - 1) == 1)
assert(number.GCD(e, phi) == 1)


e: 65537


In [33]:
# generate private key d
# Choose d the multiplicative inverse of e
d = pow(e, -1, phi)

print("d:", d)

# simple check
assert(d != 1)
assert(d != e)


d: 1957653290186730161302036633373872955919159011784792214527018358392252143651762842263283005960945269339172516138027995871191575899623666386623229741897217


In [34]:
# Encryption and Decryption
#

# The cipher c is the encrypted form of the message and is sent to the message receiver.
c=pow(M,e,n)

# Upon receipt of the cipher, the receiver decrypts the message with their private key
msg_plaintext = pow(c,d,n)
msg_decryptedtext = number.long_to_bytes(msg_plaintext).decode('utf-8')

print('\nPublic key (e,n) for the encryption is:\ne: %d\nn: %d' %(e,n))
print('\nPrivate key (d,n) for the decryption is:\nd: %d\nn: %d' %(d,n))
print('\nCipher: %s' % c)

print('\nSummery:')
print("Original message:", msg)
print("Plain message:", M)
print("Encrypted cipher message:", c)
print("Plain message agine:", msg_plaintext)
print("Decrypted message:", msg_decryptedtext)


Public key (e,n) for the encryption is:
e: 65537
n: 4941408245222913826115066046888904325684560320264209188239762831380104326702713764726239770396792159551303492824443440489374148043327378137455830566693801

Private key (d,n) for the decryption is:
d: 1957653290186730161302036633373872955919159011784792214527018358392252143651762842263283005960945269339172516138027995871191575899623666386623229741897217
n: 4941408245222913826115066046888904325684560320264209188239762831380104326702713764726239770396792159551303492824443440489374148043327378137455830566693801

Cipher: 2724113885362117836698792150647464086440402631180738061852280490077014903603576777432806784380676484285599477812863560983202419360227628103196336957524602

Summery:
Original message: this is my secret
Plain message: 39611541800605679895468651228787463316852
Encrypted cipher message: 272411388536211783669879215064746408644040263118073806185228049007701490360357677743280678438067648428559947781286356098320241936022762810319

## Code Summery

In [35]:
# Setting up the encryption
p = number.getPrime(128)
q = number.getPrime(128)

# key generation
n = p*q
phi = (p-1)*(q-1)

e = 65537
d = pow(e, -1, phi)
assert(number.GCD(d,n) == 1)

# Encryption
c=pow(M,e,n)
# Decryption
msg_plaintext = pow(c,d,n)
msg_decryptedtext = number.long_to_bytes(msg_plaintext).decode('utf-8')

print('Checks equality:', msg == msg_decryptedtext)

Checks equality: True


In [36]:
print(f'Bob uses RSA to send an encrypted message to Alice. The public exponent (e) is {e} and the modulus (N) is {n}.\nWith a cipher of {c}, determine the decrypted message "{msg_decryptedtext}"')

print('\nPublic key (e,n) for the encryption is:\ne: %d\nn: %d' %(e,n))
print('\nPrivate key (d,n) for the decryption is:\nd: %d\nn: %d' %(d,n))
print('\nEncrypted cipher message:: %s' % c)

print("Decrypted message:", msg_decryptedtext)


Bob uses RSA to send an encrypted message to Alice. The public exponent (e) is 65537 and the modulus (N) is 72398231417887145951863177911031974803489942523732002768558701652318268332089.
With a cipher of 27196274421509523956859671322033485355297339762051313167141369352972967196143, determine the decrypted message "this is my secret"

Public key (e,n) for the encryption is:
e: 65537
n: 72398231417887145951863177911031974803489942523732002768558701652318268332089

Private key (d,n) for the decryption is:
d: 25011342562710878915668007554262095328038042459357760355248731586429543288705
n: 72398231417887145951863177911031974803489942523732002768558701652318268332089

Encrypted cipher message:: 27196274421509523956859671322033485355297339762051313167141369352972967196143
Decrypted message: this is my secret


## Code examples links
- https://asecuritysite.com/rsa/rsa_full
- https://asecuritysite.com/rsa/rsa_crt4
- https://asecuritysite.com/rsa/rsa_ctf04
- https://asecuritysite.com/rsa/rsa_ctf05
- https://asecuritysite.com/rsa/rsa_12_2
- https://asecuritysite.com/cracking/rsa_ctf04
- https://mathybit.github.io/crypto-rsa/
- https://ctftime.org/writeup/29741
- https://cryptobook.nakov.com/asymmetric-key-ciphers/rsa-encrypt-decrypt-examples
- https://crypto.stackexchange.com/questions/12255/in-rsa-why-is-it-important-to-choose-e-so-that-it-is-coprime-to-%CF%86n
- https://en.wikipedia.org/wiki/RSA_%28cryptosystem%29